# Sistema de Recomendacion
El sistema de recomendacion de Machine Learning se se basa en una relacion item-item, significa que toma un item (nombre de un juego) y en base a que similar es ese item a otros, recomienda juegos similares. Para la recomendacion se utiliza similitud del csoeno.

In [1]:
# Comenzamos importando las librerias
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
# Creamos nuestro data frame con el csv de reviews
recomendacion = pd.read_csv('./archivos_csv/reviews_final.csv')

In [3]:
# Verificamos que este bien
recomendacion.head()

,user_id,item_id,recommend,review,sentiment_analysis,year
0,76561197970982479,1250.0,True,Simple yet with great replayability. In my opi...,2,2011
1,js41637,22200.0,True,It's unique and worth a playthrough.,2,2011
2,evcentric,43110.0,True,Great atmosphere. The gunplay can be a bit chu...,2,2011
3,doctr,251610.0,True,I know what you think when you see this title ...,2,2014
4,maplemage,227300.0,True,For a simple (it's actually not all that simpl...,2,2013


In [9]:
# Pedimos un info para ver los datos y tipos de datos
recomendacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20356 entries, 0 to 20355
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             20356 non-null  object 
 1   item_id             20356 non-null  float64
 2   recommend           20356 non-null  bool   
 3   review              20356 non-null  object 
 4   sentiment_analysis  20356 non-null  int64  
 5   year                20356 non-null  int64  
dtypes: bool(1), float64(1), int64(2), object(2)
memory usage: 815.2+ KB


In [13]:
# Como vamos a necesitar un item_name o titulo del juego, traemos el datadrame de items y le vamos aplicar un Merge.
items = pd.read_csv('./Archivos_csv/items_final.csv')
items.head()


,user_id,items_count,steam_id,item_id,item_name,playtime_forever
0,76561197970982479,277.0,76561197970982479,10,Counter-Strike,6
1,js41637,888.0,76561198035864385,20,Team Fortress Classic,0
2,evcentric,137.0,76561198007712555,30,Day of Defeat,7
3,Riot-Punch,328.0,76561197963445855,40,Deathmatch Classic,0
4,doctr,541.0,76561198002099482,50,Half-Life: Opposing Force,0


In [14]:
# Hacemos un merge de los 2 df por medio de user_id
data_recommend = items.merge(recomendacion, on="user_id", how="inner")

In [15]:
# Vemos si queda bien el df con el merge
data_recommend.head()

,user_id,items_count,steam_id,item_id_x,item_name,playtime_forever,item_id_y,recommend,review,sentiment_analysis,year
0,76561197970982479,277.0,76561197970982479,10,Counter-Strike,6,1250.0,True,Simple yet with great replayability. In my opi...,2,2011
1,js41637,888.0,76561198035864385,20,Team Fortress Classic,0,22200.0,True,It's unique and worth a playthrough.,2,2011
2,evcentric,137.0,76561198007712555,30,Day of Defeat,7,43110.0,True,Great atmosphere. The gunplay can be a bit chu...,2,2011
3,doctr,541.0,76561198002099482,50,Half-Life: Opposing Force,0,251610.0,True,I know what you think when you see this title ...,2,2014
4,maplemage,629.0,76561198026584251,300,Day of Defeat: Source,4733,227300.0,True,For a simple (it's actually not all that simpl...,2,2013


In [16]:
# Eliminamos nulos por si llegan a existir
data_recommend = data_recommend.dropna()

In [17]:

data_recommend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20855 entries, 0 to 20854
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             20855 non-null  object 
 1   items_count         20855 non-null  float64
 2   steam_id            20855 non-null  int64  
 3   item_id_x           20855 non-null  int64  
 4   item_name           20855 non-null  object 
 5   playtime_forever    20855 non-null  int64  
 6   item_id_y           20855 non-null  float64
 7   recommend           20855 non-null  bool   
 8   review              20855 non-null  object 
 9   sentiment_analysis  20855 non-null  int64  
 10  year                20855 non-null  int64  
dtypes: bool(1), float64(2), int64(5), object(3)
memory usage: 1.6+ MB


In [18]:
# Eliminamos las columnas que creemos que no vamos a utilizar
data_recommend = data_recommend.drop(columns=['items_count', 'playtime_forever', 'item_id_x', 'steam_id', 'sentiment_analysis', 'year'])

In [19]:
data_recommend.head()

,user_id,item_name,item_id_y,recommend,review
0,76561197970982479,Counter-Strike,1250.0,True,Simple yet with great replayability. In my opi...
1,js41637,Team Fortress Classic,22200.0,True,It's unique and worth a playthrough.
2,evcentric,Day of Defeat,43110.0,True,Great atmosphere. The gunplay can be a bit chu...
3,doctr,Half-Life: Opposing Force,251610.0,True,I know what you think when you see this title ...
4,maplemage,Day of Defeat: Source,227300.0,True,For a simple (it's actually not all that simpl...


In [21]:
# Renombramos columnas de item_id
data_recommend = data_recommend.rename(columns={'item_id_y': 'itema_id'})

In [22]:
data_recommend.head()

,user_id,item_name,itema_id,recommend,review
0,76561197970982479,Counter-Strike,1250.0,True,Simple yet with great replayability. In my opi...
1,js41637,Team Fortress Classic,22200.0,True,It's unique and worth a playthrough.
2,evcentric,Day of Defeat,43110.0,True,Great atmosphere. The gunplay can be a bit chu...
3,doctr,Half-Life: Opposing Force,251610.0,True,I know what you think when you see this title ...
4,maplemage,Day of Defeat: Source,227300.0,True,For a simple (it's actually not all that simpl...


In [4]:
# Inicializar el vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer()

In [23]:
# Aplicar el vectorizador a la columna 'review'
tfidf_matrix = tfidf_vectorizer.fit_transform(data_recommend['review'])

In [24]:
# Calcular la similitud del coseno entre los juegos
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [33]:
# Función para obtener recomendaciones
def get_recommendations(item_name, cosine_sim=cosine_sim):
    idx_list = data_recommend.index[data_recommend['item_name'] == item_name].tolist()
    
    if idx_list:
        idx = idx_list[0]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:6]  # Recomendar los 5 ítems más similares (excluyendo el propio ítem)
        game_indices = [i[0] for i in sim_scores]
        return data_recommend['item_name'].iloc[game_indices]
    else:
        return []  # Manejar el caso en el que el ítem no se encontró


In [34]:
# Obtener recomendaciones para un juego específico
item_name = 'Counter-Strike'
recommendations = get_recommendations(item_name)
print(f"Recomendaciones para '{item_name}':")
print(recommendations)

Recomendaciones para 'Counter-Strike':
15740                                     Trove
9524              Sam & Max 301: The Penal Zone
4743                                      Depth
11684                               Half-Life 2
3520     Tom Clancy's Ghost Recon Phantoms - NA
18874                    Planetary Annihilation
6326                     Medieval II: Total War
20493                             Saints Row IV
14761          Arma 2: Private Military Company
6157              resident evil 4 / biohazard 4
Name: item_name, dtype: object


In [29]:
# Exportamos el df a un archivo parquet para que podamos utilzar en fastapi
ruta_parquet = ('recomendacion.parquet')

In [30]:
data_recommend.to_parquet(ruta_parquet, engine='fastparquet')